In [179]:
from pathlib import Path
from typing import List, Tuple
from tempfile import TemporaryDirectory

import pandas as pd
from spacy import displacy

from meddocan.data import meddocan_zip, ArchiveFolder
from meddocan.data.containers import BratAnnotations, BratFilesPair, BratSpan
from meddocan.data.docs_iterators import GsDocs
from meddocan.language.pipeline import meddocan_pipeline
from meddocan.data import meddocan_url, meddocan_zip, ArchiveFolder
from meddocan.data.docs_iterators import BratAnnotations, GsDocs
from meddocan.data.utils import set_ents_from_brat_spans
from meddocan.data.corpus import MEDDOCAN

# MEDDOCAN: Anonimización aplicada al ámbito médico

## Introducción

* Medical Document Anonymization Track (track 9 of the <abbr title="Iberian Languages Evaluation Forum 2019"> [IberLEF 2019](http://ceur-ws.org/Vol-2421/)).</abbr>.

    Detectar automaticamente la información sanitaria protegida (PHI)

* ¿Como?

    Name Entity Recognition con methodos de Transfer Learning basados en los transformadores


## Dataset

* Visualización de los datos

![Figure 1: An example of MEDDOCAN annotation visualized using the BRAT annotation interface.](https://temu.bsc.es/meddocan/wp-content/uploads/2019/03/image-1-1024x922.png)

* Distribución del tipo de entidad entre los juegos de datos

|               Tipo               | Train | Dev | Test | Total |
| :------------------------------: | :---: | :-: | :--: | :---: |
|            TERRITORIO            | 1875  | 987 | 956  | 3818  |
|              FECHAS              | 1231  | 724 | 611  | 2566  |
|      EDAD SUJETO ASISTENCIA      | 1035  | 521 | 518  | 2074  |
|     NOMBRE SUJETO ASISTENCIA     | 1009  | 503 | 502  | 2014  |
|    NOMBRE PERSONAL SANITARIO     | 1000  | 497 | 501  | 1998  |
|      SEXO SUJETO ASISTENCIA      |  925  | 455 | 461  | 1841  |
|              CALLE               |  862  | 434 | 413  | 1709  |
|               PAIS               |  713  | 347 | 363  | 1423  |
|       ID SUJETO ASISTENCIA       |  567  | 292 | 283  | 1142  |
|        CORREO ELECTRONICO        |  469  | 241 | 249  |  959  |
| ID TITULACION PERSONAL SANITARIO |  471  | 226 | 234  |  931  |
|         ID ASEGURAMIENTO         |  391  | 194 | 198  |  783  |
|             HOSPITAL             |  255  | 140 | 130  |  525  |
|   FAMILIARES SUJETO ASISTENCIA   |  243  | 92  |  81  |  416  |
|           INSTITUCION            |  98   | 72  |  67  |  237  |
|     ID CONTACTO ASISTENCIAL      |  77   | 32  |  39  |  148  |
|         NUMERO TELEFONO          |  58   | 25  |  26  |  109  |
|            PROFESION             |  24   |  4  |  9   |  37   |
|            NUMERO FAX            |  15   |  6  |  7   |  28   |
|     OTROS SUJETO ASISTENCIA      |   9   |  6  |  7   |  22   |
|           CENTRO SALUD           |   6   |  2  |  6   |  14   |
|   ID EMPLEO PERSONAL SANITARIO   |   0   |  1  |  0   |   1   |
| IDENTIF VEHICULOS NRSERIE PLACAS |   0   |  0  |  0   |   0   |
|   IDENTIF DISPOSITIVOS NRSERIE   |   0   |  0  |  0   |   0   |
|     NUMERO BENEF PLAN SALUD      |   0   |  0  |  0   |   0   |
|             URL WEB              |   0   |  0  |  0   |   0   |
|       DIREC PROT INTERNET        |   0   |  0  |  0   |   0   |
|        IDENTF BIOMETRICOS        |   0   |  0  |  0   |   0   |
|       OTRO NUMERO IDENTIF        |   0   |  0  |  0   |   0   |

* **Código**

El paquete ``meddocan.data`` proporciona clases y funciones para tratar con los conjuntos de datos originales de
meddocan en la web.

- `meddocan.data.meddocan_url` contiene el enlace url para llegar a las carpetas de datos comprimidas.

In [22]:
meddocan_url

MeddocanUrl(sample='http://temu.bsc.es/meddocan/wp-content/uploads/2019/03/sample-set.zip', train='http://temu.bsc.es/meddocan/wp-content/uploads/2019/03/train-set.zip', dev='http://temu.bsc.es/meddocan/wp-content/uploads/2019/04/dev-set-1.zip', test='http://temu.bsc.es/meddocan/wp-content/uploads/2019/05/test-set.zip', background='http://temu.bsc.es/meddocan/wp-content/uploads/2019/10/background-set.zip', base='http://temu.bsc.es/meddocan/wp-content/uploads/2019')

* `meddocan.data.meddocan_zip` obtiene el par de archivos brat para
una carpeta determinada del conjunto de datos en caché a través de su método ``brat_files``.

In [21]:
brat_files_pairs = meddocan_zip.brat_files(ArchiveFolder.train)

In [41]:
brat_files_pair = next(brat_files_pairs)

In [42]:
brat_files_pair.txt

Path('/home/wave/.meddocan/datasets/meddocan/train-set.zip', 'train/brat/S0004-06142005000900016-1.txt')

In [43]:
brat_files_pair.ann

Path('/home/wave/.meddocan/datasets/meddocan/train-set.zip', 'train/brat/S0004-06142005000900016-1.ann')

Visualizamos los datos brutos

In [44]:
brat_annotations = BratAnnotations.from_brat_files(brat_files_pair)

- El texto

In [75]:
text = brat_annotations.text; text

'\ufeffNombre: Valeria.\nApellidos: Roca Caballero .\nCIPA: nhc-120978.\nNASS: 21 98731245 98.\nDomicilio: Calle de la bola 19, 11A.\nLocalidad/ Provincia: Mostoles Madrid.\nCP: 28935.\n       NHC: 120978.\nDatos asistenciales.\nFecha de nacimiento: 28-05-1989.\nPaís: Colombia.\nEdad: 29 Sexo: M.\nFecha de Ingreso: 1/12/2017.\nEspecialidad: Oncología.\nMédico: Marrupe González    NºCol: 28-28-36541.\nMotivo de ingreso: Dolor en fosa renal derecha compatible con crisis renoureteral.\nAntecedentes: ulcus duodenal y estreñimiento. No antecedentes de nefrolitiasis ni hematuria ni infecciones del tracto urinario. \nResumen de pruebas complementarias: En la exploración sólo destaca una puñopercusión renal derecha positiva. La ecografía objetiva ectasia pielocalicial renal derecha con adelgazamiento del parénquima. La UIV muestra una anulación funcional de la unidad renal derecha siendo normal el resto de la exploración. Una pielografía retrograda muestra estenosis en la unión pieloureteral d

Las anotaciones

In [112]:
brat_spans = brat_annotations.brat_spans; annotations

[BratSpan(id='T1', entity_type='FECHAS', start=299, end=308, text='1/12/2017'),
 BratSpan(id='T2', entity_type='CORREO_ELECTRONICO', start=2028, end=2058, text='dmarrupe.hmtl@salud.madrid.org'),
 BratSpan(id='T3', entity_type='TERRITORIO', start=2020, end=2026, text='Madrid'),
 BratSpan(id='T4', entity_type='TERRITORIO', start=2010, end=2018, text='Móstoles'),
 BratSpan(id='T5', entity_type='TERRITORIO', start=2004, end=2009, text='28935'),
 BratSpan(id='T6', entity_type='CALLE', start=1987, end=2001, text='Río Júcar, s/n'),
 BratSpan(id='T7', entity_type='HOSPITAL', start=1958, end=1986, text='Hospital General de Móstoles'),
 BratSpan(id='T8', entity_type='NOMBRE_PERSONAL_SANITARIO', start=1924, end=1940, text='Marrupe González'),
 BratSpan(id='T9', entity_type='ID_TITULACION_PERSONAL_SANITARIO', start=370, end=381, text='28-28-36541'),
 BratSpan(id='T10', entity_type='NOMBRE_PERSONAL_SANITARIO', start=343, end=359, text='Marrupe González'),
 BratSpan(id='T11', entity_type='SEXO_SUJET

### Preprocessamiento

Para empezar llamamos nuestro pipeline ``meddocan.language.pipeline.meddocan_pipeline`` hecho con [spaCY](https://spacy.io/) y miramos sus componentes.

In [89]:
nlp = meddocan_pipeline()
pd.DataFrame(nlp.pipe_names, columns=["componentes"]).T

,0,1,2,3
componentes,missaligned_splitter,line_sentencizer,predictor,write_methods


Vamos a hacer pasar nuestro texto de ejemplo por el ``meddocan_pipeline`` y ver el efecto de cada componente.

In [90]:
doc = nlp(text)

#### Tokenización

In [163]:
pd.DataFrame([token.text for token in doc[:10]], columns=["token"]).T

,0,1,2,3,4,5,6,7,8,9
token,﻿Nombre,:,Valeria,.,\n,Apellidos,:,Roca,Caballero,.


#### Casos especiales

In [100]:
[token for token in nlp("DREnric Lopez")]

[DR, Enric, Lopez]

#### Partición del documento en párafos

In [107]:
pd.DataFrame([sent.text for sent in doc.sents][:4], columns=["párafo"])


,párafo
0,﻿Nombre: Valeria.\n
1,Apellidos: Roca Caballero .\n
2,CIPA: nhc-120978.\n
3,NASS: 21 98731245 98.\n


#### predictor y entidades

En una primera fase no tenemos modelos con el cual hacer predicciones.
Entonces añadimos la entidades a nuestro objeto ``doc`` usando la function ``meddocan.data.utils.set_ents_from_brat_spans``.

In [128]:
doc = set_ents_from_brat_spans(doc, brat_spans=brat_spans)
displacy.render(doc[:100], style="ent")

#### Write methods

Para la evaluación, se puede escribir los datos al formato **Brat**

In [155]:
with TemporaryDirectory() as td:
    pth = Path(td, "file.txt")
    doc._.to_ann(pth)
    for i, line in enumerate(pth.read_text().split("\n")):
        print(line)
        if i > 10:
            break

T_0	NOMBRE_SUJETO_ASISTENCIA 9 16	Valeria
T_1	NOMBRE_SUJETO_ASISTENCIA 29 43	Roca Caballero
T_2	ID_SUJETO_ASISTENCIA 56 62	120978
T_3	ID_ASEGURAMIENTO 70 84	21 98731245 98
T_4	CALLE 97 121	Calle de la bola 19, 11A
T_5	TERRITORIO 145 153	Mostoles
T_6	TERRITORIO 154 160	Madrid
T_7	TERRITORIO 166 171	28935
T_8	ID_SUJETO_ASISTENCIA 185 191	120978
T_9	FECHAS 235 245	28-05-1989
T_10	PAIS 253 261	Colombia
T_11	EDAD_SUJETO_ASISTENCIA 269 271	29


Para el entrenamiento, los datos se escriben al formato **conll03**.

In [154]:
with TemporaryDirectory() as td:
    pth = Path(td, "file.txt")
    doc._.to_connl03(pth)
    for i, line in enumerate(pth.read_text().split("\n")):
        print(line)
        if i > 10:
            break

Nombre O
: O
Valeria B-NOMBRE_SUJETO_ASISTENCIA
. O

Apellidos O
: O
Roca B-NOMBRE_SUJETO_ASISTENCIA
Caballero I-NOMBRE_SUJETO_ASISTENCIA
. O

CIPA O


## Entrenamiento con Flair

### Creación del dataset ``MEDDOCAN``

Los distinctos conjunto de datos

In [174]:
pd.DataFrame([folder for folder in ArchiveFolder], columns=["sets"])

,sets
0,ArchiveFolder.train
1,ArchiveFolder.dev
2,ArchiveFolder.test
3,ArchiveFolder.sample
4,ArchiveFolder.background


Solo se usan los sets de *train*, *dev* y *test*.

Utilizamos el objecto ``meddocan.data.docs_iterators.GsDocs`` que nos permitte obtener objectos ``Doc`` usando el ``meddocan_pipeline`` para cada una de los conjuntos de datos.

In [176]:
gs_docs = GsDocs(ArchiveFolder.train)
docs_with_brat_pair = iter(gs_docs)
doc_with_brat_pair = next(docs_with_brat_pair)

``doc_with_brat_pair`` ayuda a crear el ``MEDDOCAN`` corpus que hereda de ``flair.datasets.ColumnCorpus``.

El ``MEDDOCAN`` corpus esta creado a partir de ficheros temporales que contienen el texto asi como los offsets al formato **connl03**.

In [180]:
corpus = MEDDOCAN(sentences=True, in_memory=True, document_separator_token="-DOCSTART-")

2023-02-24 12:56:51,190 Reading data from /tmp/tmp3jida_9s
2023-02-24 12:56:51,191 Train: /tmp/tmp3jida_9s/train
2023-02-24 12:56:51,192 Dev: /tmp/tmp3jida_9s/dev
2023-02-24 12:56:51,192 Test: /tmp/tmp3jida_9s/test


In [181]:
print(corpus)

Corpus: 10811 train + 5518 dev + 5405 test sentences


Ahora podemos pasar al entrenamiento!

### Transfert learning

Comparación entre el aprendizaje supervisado tradicional (izquierda) y el transfer learning (derecha)

![transfer-learning](../cdti/figures/transformers-1.png)

### Flert

![Flert](../cdti/figures/flert-1.png)

* **baseline**

    * Flair + LSTM +CRF

* **2 enfoques para el NER basados en Transformers**

    * Finetuning + linear transformation
    * Caractéristica + LSTM + CRF



### Entrenamiento

## Evaluación

In [63]:
gs_docs = GsDocs(ArchiveFolder.train)
docs_with_brat_pair = iter(gs_docs)
doc_with_brat_pair = next(docs_with_brat_pair)